2024

In [ ]:
import pandas as pd
import numpy as np

# Carica il file Excel (sostituisci con il nome corretto)
file = r"..\ROTTURE\R-interventi-2024_gen21_chiusura anno.xlsx"  # Cambia questo con il nome reale del file
df = pd.read_excel(file, sheet_name='base')

# Seleziona le colonne chiave
col_modello = 'Modello'
col_data_acquisto = 'Data Acquisto'
col_data_apertura = 'Data Apertura'
col_ricambi = ['Ricambio', 'Ricambio.1', 'Ricambio.2', 'Ricambio.3', 'Ricambio.4']

# Prepara una lista per le righe del nuovo dataframe
records = []

for idx, row in df.iterrows():
    modello = row[col_modello]
    data_acquisto = pd.to_datetime(row[col_data_acquisto], errors='coerce')
    data_apertura = pd.to_datetime(row[col_data_apertura], errors='coerce')
    tempo_di_vita = (data_apertura - data_acquisto).days if pd.notna(data_apertura) and pd.notna(data_acquisto) else np.nan
    
    for col in col_ricambi:
        val = row.get(col)
        if pd.isna(val):
            continue
        try:
            qty, codice, descrizione = str(val).split('|', maxsplit=2)
            records.append({
                'Modello': modello,
                'Data Acquisto': data_acquisto,
                'Data Apertura': data_apertura,
                'Tempo di Vita (giorni)': tempo_di_vita,
                'Quantità': int(qty.strip()),
                'Codice Componente': codice.strip(),
                'Descrizione Componente': descrizione.strip()
            })
        except ValueError:
            # Gestione valori mal formattati
            continue

# Crea il nuovo DataFrame
df_rotture = pd.DataFrame(records)

# Visualizza le prime righe
print(df_rotture.head())


2023

In [ ]:
import pandas as pd
import numpy as np
import math

# Carica il file Excel
file = r'..\ROTTURE\R-Interventi-2023_gen23_chiusura anno.xlsx'  # Sostituisci con il file reale
df = pd.read_excel(file, sheet_name='2023')  # Foglio visibile in basso

# Colonne di interesse
col_modello = 'Modello'
col_data_acquisto = 'Data Acquisto'
col_data_apertura = 'Data Apertura'
col_pezzi = 'Nr Pezzi'
col_ricambi_count = 'Nr Ricambi'
col_ricambi = ['Ricambio', 'Ricambio.1', 'Ricambio.2', 'Ricambio.3', 'Ricambio.4']

records = []

for idx, row in df.iterrows():
    modello = row.get(col_modello)
    data_acquisto = pd.to_datetime(row.get(col_data_acquisto), errors='coerce')
    data_apertura = pd.to_datetime(row.get(col_data_apertura), errors='coerce')
    tempo_di_vita = (data_apertura - data_acquisto).days if pd.notna(data_apertura) and pd.notna(data_acquisto) else np.nan

    try:
        nr_pezzi = int(row.get(col_pezzi))
        nr_ricambi = int(row.get(col_ricambi_count))
        qty_per_ricambio = math.ceil(nr_pezzi / nr_ricambi) if nr_ricambi else np.nan
    except (ValueError, TypeError):
        qty_per_ricambio = np.nan

    for col in col_ricambi:
        descrizione = row.get(col)
        if pd.notna(descrizione) and str(descrizione).strip() != '':
            records.append({
                'Modello': modello,
                'Data Acquisto': data_acquisto,
                'Data Apertura': data_apertura,
                'Tempo di Vita (giorni)': tempo_di_vita,
                'Quantità': qty_per_ricambio,
                'Codice Componente': np.nan,
                'Descrizione Componente': str(descrizione).strip()
            })

# Crea il nuovo DataFrame
df_rotture = pd.DataFrame(records)

# Esporta se serve
# df_rotture.to_excel('rotture_componenti_2023.xlsx', index=False)

print(df_rotture.head())


In [ ]:
df_rotture

2022

In [ ]:
import pandas as pd
import numpy as np

# Carica il file
file = r'..\ROTTURE\R-interventi-2022_gen17_chiusura anno.xlsx'
df = pd.read_excel(file, sheet_name='2022')

# Colonne rilevanti
col_modello = 'Mod.'
col_data_acquisto = 'Data Acquisto'
col_data_apertura = 'Data Apertura'
col_codici = 'Codici Ricambi'
col_quantita = 'QT'
col_descrizione = 'Descrizioni Ricambi'

records = []

for idx, row in df.iterrows():
    modello = row.get(col_modello)
    data_acquisto = pd.to_datetime(row.get(col_data_acquisto), errors='coerce')
    data_apertura = pd.to_datetime(row.get(col_data_apertura), errors='coerce')
    tempo_di_vita = (data_apertura - data_acquisto).days if pd.notna(data_apertura) and pd.notna(data_acquisto) else np.nan
    descrizione = str(row.get(col_descrizione)).strip() if pd.notna(row.get(col_descrizione)) else None

    # Ottieni lista dei codici separati da '|'
    codici_raw = row.get(col_codici)
    if pd.isna(codici_raw):
        continue
    codici = [c.strip() for c in str(codici_raw).split('|') if c.strip()]

    try:
        qty = int(row.get(col_quantita))
    except (ValueError, TypeError):
        qty = np.nan

    for codice in codici:
        records.append({
            'Modello': modello,
            'Data Acquisto': data_acquisto,
            'Data Apertura': data_apertura,
            'Tempo di Vita (giorni)': tempo_di_vita,
            'Quantità': qty,
            'Codice Componente': codice,
            'Descrizione Componente': descrizione
        })

# Crea il DataFrame finale
df_rotture = pd.DataFrame(records)

# Esporta opzionalmente
# df_rotture.to_excel('rotture_componenti_2022.xlsx', index=False)

print(df_rotture.head())


In [ ]:
df_rotture

ROTTURE

In [2]:
import pandas as pd
import numpy as np
import math

# ----------- Funzione utilità: converte date in formato gg/mm/aaaa -----------
def parse_and_format_date(date_val):
    try:
        dt = pd.to_datetime(date_val, errors='coerce', dayfirst=True)
        if pd.notna(dt):
            return dt.strftime('%d/%m/%Y')
    except Exception:
        pass
    return np.nan

# ----------- FUNZIONE 1: Elabora file 2024 -----------
def process_file_2024(path):
    df = pd.read_excel(path, sheet_name='base')
    col_ricambi = ['Ricambio', 'Ricambio.1', 'Ricambio.2', 'Ricambio.3', 'Ricambio.4']
    records = []

    for _, row in df.iterrows():
        modello = row.get('Modello')
        data_acquisto = parse_and_format_date(row.get('Data Acquisto'))
        data_apertura = parse_and_format_date(row.get('Data Apertura'))

        # Calcolo tempo di vita in giorni
        t1 = pd.to_datetime(data_acquisto, format='%d/%m/%Y', errors='coerce')
        t2 = pd.to_datetime(data_apertura, format='%d/%m/%Y', errors='coerce')
        tempo_di_vita = (t2 - t1).days if pd.notna(t1) and pd.notna(t2) else np.nan

        for col in col_ricambi:
            val = row.get(col)
            if pd.isna(val):
                continue
            try:
                qty, codice, descrizione = str(val).split('|', maxsplit=2)
                records.append({
                    'Modello': modello,
                    'Data Acquisto': data_acquisto,
                    'Data Apertura': data_apertura,
                    'Tempo di Vita (giorni)': tempo_di_vita,
                    'Quantità': int(qty.strip()),
                    'Codice Componente': codice.strip(),
                    'Descrizione Componente': descrizione.strip()
                })
            except Exception:
                continue
    return pd.DataFrame(records)

# ----------- FUNZIONE 2: Elabora file 2023 -----------
def process_file_2023(path):
    df = pd.read_excel(path, sheet_name='2023')
    col_ricambi = ['Ricambio', 'Ricambio.1', 'Ricambio.2', 'Ricambio.3', 'Ricambio.4']
    records = []

    for _, row in df.iterrows():
        modello = row.get('Modello')
        data_acquisto = parse_and_format_date(row.get('Data Acquisto'))
        data_apertura = parse_and_format_date(row.get('Data Apertura'))

        t1 = pd.to_datetime(data_acquisto, format='%d/%m/%Y', errors='coerce')
        t2 = pd.to_datetime(data_apertura, format='%d/%m/%Y', errors='coerce')
        tempo_di_vita = (t2 - t1).days if pd.notna(t1) and pd.notna(t2) else np.nan

        try:
            nr_pezzi = int(row.get('Nr Pezzi'))
            nr_ricambi = int(row.get('Nr Ricambi'))
            qty_per_ricambio = math.ceil(nr_pezzi / nr_ricambi) if nr_ricambi else np.nan
        except Exception:
            qty_per_ricambio = np.nan

        for col in col_ricambi:
            descrizione = row.get(col)
            if pd.notna(descrizione) and str(descrizione).strip() != '':
                records.append({
                    'Modello': modello,
                    'Data Acquisto': data_acquisto,
                    'Data Apertura': data_apertura,
                    'Tempo di Vita (giorni)': tempo_di_vita,
                    'Quantità': qty_per_ricambio,
                    'Codice Componente': np.nan,
                    'Descrizione Componente': str(descrizione).strip()
                })
    return pd.DataFrame(records)

# ----------- FUNZIONE 3: Elabora file 2022 -----------
def process_file_2022(path):
    df = pd.read_excel(path, sheet_name='2022')
    records = []

    for _, row in df.iterrows():
        modello = row.get('Mod.')
        data_acquisto = parse_and_format_date(row.get('Data Acquisto'))
        data_apertura = parse_and_format_date(row.get('Data Apertura'))

        t1 = pd.to_datetime(data_acquisto, format='%d/%m/%Y', errors='coerce')
        t2 = pd.to_datetime(data_apertura, format='%d/%m/%Y', errors='coerce')
        tempo_di_vita = (t2 - t1).days if pd.notna(t1) and pd.notna(t2) else np.nan

        codici_raw = row.get('Codici Ricambi')
        descrizioni_raw = row.get('Descrizioni Ricambi')

        if pd.isna(codici_raw):
            continue

        codici = [c.strip() for c in str(codici_raw).split('|') if c.strip()]
        descrizioni = [d.strip() for d in str(descrizioni_raw).split('|')] if pd.notna(descrizioni_raw) else [''] * len(codici)
        if len(descrizioni) < len(codici):
            descrizioni += [''] * (len(codici) - len(descrizioni))

        try:
            qty = int(row.get('QT'))
        except Exception:
            qty = np.nan

        for codice, descrizione in zip(codici, descrizioni):
            records.append({
                'Modello': modello,
                'Data Acquisto': data_acquisto,
                'Data Apertura': data_apertura,
                'Tempo di Vita (giorni)': tempo_di_vita,
                'Quantità': qty,
                'Codice Componente': codice,
                'Descrizione Componente': descrizione
            })
    return pd.DataFrame(records)

# ----------- ELABORA TUTTI I FILE -----------
df_2024 = process_file_2024(r'..\ROTTURE\R-interventi-2024_gen21_chiusura anno.xlsx')
df_2023 = process_file_2023(r'..\ROTTURE\R-Interventi-2023_gen23_chiusura anno.xlsx')
df_2022 = process_file_2022(r'..\ROTTURE\R-interventi-2022_gen17_chiusura anno.xlsx')

# Unisci i dataframe
df_rotture = pd.concat([df_2024, df_2023, df_2022], ignore_index=True)

# Esporta se desiderato
# df_rotture.to_excel('rotture_unificate.xlsx', index=False)

# Mostra le prime righe
print(df_rotture.head())


      Modello Data Acquisto Data Apertura  Tempo di Vita (giorni)  Quantità  \
0      DSW610    23/07/2022    02/01/2024                   528.0         1   
1      DSW610    23/07/2022    02/01/2024                   528.0         1   
2  DHLB7012SL    29/10/2022    02/01/2024                   430.0         1   
3   TK1M6AL8K    19/07/2023    02/01/2024                   167.0         1   
4   TK1M6AL8K    19/07/2023    02/01/2024                   167.0         1   

    Codice Componente               Descrizione Componente  
0  X04-F60118UU000001                    CESTELLO COMPLETO  
1      X2303-001-5091  SEMIVASCA POSTERIORE CON CUSCINETTI  
2            H1992064            POMPA DI SCARICO + FILTRO  
3         021.05.1007              LEVA DEL FRENO SINISTRA  
4         021.05.1015                         ACCELERATORE  


In [4]:
df_rotture.to_excel('output_rotture.xlsx')

In [17]:
import pandas as pd

# Carica file rotture e spareorder
rotture = pd.read_excel("output_rotture.xlsx")
spareorder = pd.read_excel("output_spareorder.xlsx")

# Identifica il nome della colonna modello nei due file
print("Colonne rotture:", rotture.columns)
print("Colonne spareorder:", spareorder.columns)

# Estrai i modelli validi dallo spareorder (senza null)
modelli_validi = spareorder['modello'].dropna().unique()

# Filtra solo rotture con modello presente in spareorder
rotture_filtrate = rotture[rotture['Modello'].isin(modelli_validi)]

len(rotture_filtrate)

rotture_filtrate.to_excel('output_rotture_filtrate.xlsx')

Colonne rotture: Index(['Unnamed: 0', 'Modello', 'Data Acquisto', 'Data Apertura',
       'Tempo di Vita (giorni)', 'Quantità', 'Codice Componente',
       'Descrizione Componente'],
      dtype='object')
Colonne spareorder: Index(['modello', 'header', 'data', 'valore', 'source_file'], dtype='object')


In [10]:
import pandas as pd
from difflib import SequenceMatcher

# Carica i file
df_rotture = pd.read_excel("output_rotture_filtrate.xlsx")
df_anagrafica = pd.read_excel("output_anagrafica.xlsx")

# Crea una copia del df rotture
df_finale = df_rotture.copy()

# Trova righe con codice mancante
mask_mancante = df_rotture["Codice Componente"].isnull() | (df_rotture["Codice Componente"] == '')

# Funzione per calcolare la similarità tra due stringhe
def similarita(a, b):
    return SequenceMatcher(None, str(a), str(b)).ratio()

# Per ogni riga mancante, trova il codice coerente e aggiorna
for idx, row in df_rotture[mask_mancante].iterrows():
    modello = row["Modello"]
    descr_mancante = row["Descrizione Componente"]
    df_anag_modello = df_anagrafica[df_anagrafica["modello"] == modello]

    max_score = -1
    codice_assoc = None

    for _, ana_row in df_anag_modello.iterrows():
        descr_anagrafica = ana_row["descrizione"]
        score = similarita(descr_mancante, descr_anagrafica)
        if score > max_score:
            max_score = score
            codice_assoc = ana_row["codice"]
    
    # Aggiorna il DataFrame con il codice trovato
    df_finale.at[idx, "Codice Componente"] = codice_assoc

# Ora df_finale ha la colonna 'Codice Componente' senza più vuoti (se c'era match)
print(df_finale.head())


   Unnamed: 0.1  Unnamed: 0     Modello Data Acquisto Data Apertura  \
0             2           2  DHLB7012SL    29/10/2022    02/01/2024   
1            12          12  WMHN914ASJ    27/06/2023    02/01/2024   
2            13          13  WMHN914ASJ    27/06/2023    02/01/2024   
3            26          26  WMEH147SVA    10/06/2023    02/01/2024   
4            27          27  WMEH147SVA    10/06/2023    02/01/2024   

   Tempo di Vita (giorni)  Quantità Codice Componente  \
0                     430         1          H1992064   
1                     189         1          H1933068   
2                     189         1          H2023492   
3                     206         1          H2239135   
4                     206         1          H2243793   

                         Descrizione Componente  
0                     POMPA DI SCARICO + FILTRO  
1  RESISTENZA ELETTRICA SCALDANTE + SONDA TEMP.  
2                     POMPA DI SCARICO + FILTRO  
3                             

In [18]:
# Funzione per recuperare lo stat relativo a codice e modello
def get_stat(codice, modello):
    row = df_anagrafica[(df_anagrafica["codice"] == codice) & (df_anagrafica["modello"] == modello)]
    if not row.empty:
        return row.iloc[0]["stat"]
    else:
        return None

# Applica la funzione a ogni riga del DataFrame finale
df_finale["stat"] = df_finale.apply(
    lambda r: get_stat(r["Codice Componente"], r["Modello"]), axis=1
)

# Mostra l'anteprima del nuovo DataFrame
print(df_finale.head())

   Unnamed: 0.1  Unnamed: 0     Modello Data Acquisto Data Apertura  \
0             2           2  DHLB7012SL    29/10/2022    02/01/2024   
1            12          12  WMHN914ASJ    27/06/2023    02/01/2024   
2            13          13  WMHN914ASJ    27/06/2023    02/01/2024   
3            26          26  WMEH147SVA    10/06/2023    02/01/2024   
4            27          27  WMEH147SVA    10/06/2023    02/01/2024   

   Tempo di Vita (giorni)  Quantità Codice Componente  \
0                     430         1          H1992064   
1                     189         1          H1933068   
2                     189         1          H2023492   
3                     206         1          H2239135   
4                     206         1          H2243793   

                         Descrizione Componente stat  
0                     POMPA DI SCARICO + FILTRO   L1  
1  RESISTENZA ELETTRICA SCALDANTE + SONDA TEMP.   L3  
2                     POMPA DI SCARICO + FILTRO   L1  
3         

In [19]:
mask_stat_mancante = df_finale["stat"].isnull() | (df_finale["stat"] == '')

In [21]:
df_finale[mask_stat_mancante]

,Unnamed: 0.1,Unnamed: 0,Modello,Data Acquisto,Data Apertura,Tempo di Vita (giorni),Quantità,Codice Componente,Descrizione Componente,stat
5,52,52,CFE100SH4WF1,11/12/2023,02/01/2024,22,1,FH2500300242,SCHEDA ELETTRONICA DI CONTROLLO,None
24,192,192,TFCBNF350IX,04/01/2024,04/01/2024,0,1,FH2100280967,PORTA DEL VANO FRIGORIFERO (CBV),None
27,204,204,WMEH147SVA,31/08/2022,04/01/2024,491,2,H2024630,AMMORTIZZATORE,None
28,205,205,SHCB43NM2XE0,04/01/2024,04/01/2024,0,1,FH2100281682,PORTA DEL VANO FRIGORIFERO,None
31,241,241,OFZ434F,29/09/2023,04/01/2024,97,1,FH2400100420,COMPRESSORE,None
...,...,...,...,...,...,...,...,...,...,...
10056,76130,76130,TME-640NV4XE0,27/07/2021,28/12/2022,519,2,ELM25014,scheda elettronica di controllo,None
10060,76146,76146,TME-640NV4KE1,16/08/2021,28/12/2022,499,1,TMM26020,pannello (paratia) vano freezer (DP),None
10065,76170,76170,TME-28NSM1XF0,19/03/2022,29/12/2022,285,1,TEM21011,termostato solo con sonda a bulbo,None
10069,76176,76176,TME-28NSM1XF0,14/05/2021,29/12/2022,594,1,TEM21011,termostato solo con sonda a bulbo,None


In [23]:
from difflib import SequenceMatcher

# Funzione similarità descrizioni
def similarita(a, b):
    return SequenceMatcher(None, str(a), str(b)).ratio()

# Maschera delle righe ancora senza stat
mask_stat_mancante = df_finale["stat"].isnull() | (df_finale["stat"] == '')

for idx, row in df_finale[mask_stat_mancante].iterrows():
    modello = row["Modello"]
    descr_mancante = row["Descrizione Componente"]
    # Filtra l'anagrafica per il modello
    df_anag_modello = df_anagrafica[df_anagrafica["modello"] == modello]
    # Cerca descrizione più simile
    max_score = -1
    stat_assoc = None
    for _, ana_row in df_anag_modello.iterrows():
        descr_anagrafica = ana_row["descrizione"]
        score = similarita(descr_mancante, descr_anagrafica)
        if score > max_score:
            max_score = score
            stat_assoc = ana_row["stat"]
    # Assegna stat trovato (se c'è match)
    df_finale.at[idx, "stat"] = stat_assoc

# (facoltativo) Salva il risultato
# df_finale.to_excel("output_rotture_con_stat_completo.xlsx", index=False)

# Anteprima
print(df_finale[df_finale["stat"].isnull() | (df_finale["stat"] == '')])


      Unnamed: 0.1  Unnamed: 0           Modello Data Acquisto Data Apertura  \
670           5246        5246      SDLE16SM1XE0    01/03/2024    01/03/2024   
732           5758        5758      SDLE16SM1XE0    08/03/2024    08/03/2024   
733           5759        5759      SDLE16SM1XE0    08/03/2024    08/03/2024   
850           6592        6592      SDLE16SM1XE0    20/03/2024    20/03/2024   
1838         14403       14403      SDLE16SM1XE0    03/07/2024    03/07/2024   
1969         15649       15649          TFDP2401    18/07/2024    18/07/2024   
2222         17759       17759      SDLE16SM1XE0    14/08/2024    14/08/2024   
2276         18092       18092      SDLE16SM1XE0    21/08/2024    21/08/2024   
2374         18910       18910      SDLE16SM1XE0    29/08/2024    29/08/2024   
2545         20485       20485          TFDP2401    05/09/2024    16/09/2024   
2574         20713       20713          TFDP2401    01/03/2024    17/09/2024   
2585         20851       20851      SDLE

In [24]:
len(df_finale[df_finale["stat"].isnull() | (df_finale["stat"] == '')])

42

In [32]:
df_finale.to_excel('output_rotture_filtrate_completate.xlsx')

In [34]:
len(df_finale['Modello'].unique())

497

In [35]:
len(df_finale['Codice Componente'].unique())

963

In [36]:
len(df_finale['stat'].unique())

56